In [ ]:
!pip install speechbrain -q
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import soundfile as sf

import librosa
from IPython.display import Audio
from typing import Callable, Optional, Union

import speechbrain as sb
from speechbrain.pretrained import EncoderDecoderASR

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
def freeze_model(model):
  for p in model.parameters():
    p.requires_grad = False

def get_sb_asr(device):
  asr_model = EncoderDecoderASR.from_hparams(
      source="speechbrain/asr-wav2vec2-commonvoice-en",
      savedir="pretrained_models/asr-wav2vec2-commonvoice-en",
      run_opts={"device":device}
  )# https://huggingface.co/speechbrain/asr-wav2vec2-commonvoice-en
  return asr_model

In [ ]:
os.chdir('/content/gdrive/MyDrive/MSU/code/')

In [ ]:
asr_model = get_sb_asr(device)
encoder = dict(asr_model.named_modules())['mods.encoder']
asr_model.device

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

'cuda:0'

In [ ]:
path = 'p226_001.wav'
# asr_model.transcribe_file(path)

In [ ]:
# https://github.com/speechbrain/speechbrain/blob/7510c5bf2bf7eab99bc3e852eb17b786f7bb17bb/speechbrain/pretrained/interfaces.py
# EncoderDecoderASR -> transcribe_file -> load_audio/transcribe_batch
#    load_audio -> audio_normalizer -> AudioNormalizer -> speechbrain.dataio.preprocess
#    transcribe_batch -> encode_batch
waveform, sr = torchaudio.load(path)
waveform = waveform.squeeze(0)

wavs = waveform.unsqueeze(0) # convert to batch
wav_lens = torch.tensor([1.0])

In [ ]:
wavs.shape

In [ ]:
with torch.no_grad():
    wav_lens = wav_lens.to(asr_model.device)
    wavs = wavs.float()
    wavs, wav_lens = wavs.to(asr_model.device), wav_lens.to(asr_model.device)
    encoder_out = encoder(wavs, wav_lens)
    print(encoder_out.shape)

    predicted_tokens, scores = asr_model.mods.decoder(encoder_out, wav_lens)
    predicted_words = [asr_model.tokenizer.decode_ids(token_seq)for token_seq in predicted_tokens]
    print(predicted_words)

torch.Size([1, 113, 1024])
['PLEASE CALL STELLA']


In [ ]:
(113*1024) / (128*128), 128*128, 113*1024

(7.0625, 16384, 115712)